In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('display.max_columns', 30)
import re
import math
from datetime import datetime, timedelta

In [2]:
pwd!

'/Users/suneethi/Documents/nssdatasc/dq4_nashville_metro/dq4-metro-beetles'

In [3]:
metro_trash=pd.read_excel('data/Trash Hauler Report - Nov 2017 - Nov 2019 Final.xlsx', skiprows=1)
metro=pd.read_csv('data/hubNashville__311__Service_Requests.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
metro=metro.drop(columns=['Request Origin', 'System of Origin ID', 'Contact Type', 
                              'Preferred Language', 'State Issue'])

In [5]:
metro.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,108110,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/10/18 16:14,3/5/19 21:34,NaN,False,413 Freda Villa,MADISON,8.0,37115,36.243960,-86.727210,"(36.24396, -86.72721)"
1,122078,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/7/18 14:50,12/7/18 14:50,NaN,True,"4906 Dakota Ave, Nashville, TN 37209, United S...",NASHVILLE,24.0,37209,36.147680,-86.846440,"(36.14768, -86.84644)"
2,113730,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,11/1/18 10:00,11/6/18 11:15,NaN,True,"873 Granada Ave, Nashville, TN 37206, United S...",NASHVILLE,5.0,37206,36.189230,-86.756430,"(36.18923, -86.75643)"
3,124624,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/19/18 17:47,12/19/18 17:47,NaN,False,4304 Lindawood Dr,NASHVILLE,34.0,37215,36.097154,-86.831878,"(36.0971543, -86.8318783)"
4,108241,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,10/10/18 14:49,10/24/18 16:27,NaN,True,"336 Blackman Rd, Nashville, TN 37211, United S...",NASHVILLE,26.0,37211,36.074580,-86.746960,"(36.07458, -86.74696)"


In [6]:
metro=metro.rename(columns={'Request #': 'Request_id', 'Request Type':'Request_type',
                                'Subrequest Type': 'Subrequest_type', 
                                'Additional Subrequest Type': 'Additional_subrequest',
                               'Date / Time Opened' :'dt_opened', 'Date / Time Closed':'dt_closed',
                               'Council District': 'Council_dist', 'ZIP':'Zip', 'Latitude':'Lat', 
                               'Longitude':'Long', 'Mapped Location': 'Location'})

In [7]:
metro_misd=metro[(metro['Subrequest_type']=='Missed Pickup - Recycle Cart Service') | (metro['Subrequest_type']==
                                                                                'Missed Pickup - Trash Cart Service')
         | (metro['Subrequest_type']=='Missed Pickup - Cart Service') | (metro['Subrequest_type']=='Missed Pickup - Dumpster Service')]

In [8]:
metro_trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


keywords = ['miss', 'missed']
keywords_required = ['miss', 'missed']
#keywords_not_required = ['Damage to Property']

def check_keywords(df, keywords):
    contains_keywords = pd.DataFrame()
    for keyword in keywords:
        contains_keyword = pd.DataFrame()
        for column in df:
            if df[column].dtype not in ['float64', 'int64']:
                contains_keyword[column] = df[column].str.lower().str.contains(keyword.lower())
        contains_keywords[keyword] = contains_keyword.any(axis=1)
    return contains_keywords.astype('int64')

keyword_scores = check_keywords(metro_trash, keywords)
keyword_required_score = keyword_scores[keywords_required].sum(axis=1)
#keyword_not_required_score = keyword_scores[keywords_not_required].sum(axis=1)

metro_keyword_scores = pd.concat([metro_trash['Request'], keyword_required_score], axis=1)
metro_keyword_scores.columns = ['Request', 'keyword_required_score']
metro_keywd = pd.merge(metro_trash, metro_keyword_scores)


In [9]:
metro_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
Request Number      20226 non-null int64
Date Opened         20226 non-null datetime64[ns]
Request             20226 non-null object
Description         20197 non-null object
Incident Address    20217 non-null object
Zip Code            20151 non-null float64
Trash Hauler        19325 non-null object
Trash Route         19279 non-null object
Council District    20177 non-null float64
State Plan X        20198 non-null float64
State Plan Y        20198 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 1.7+ MB


In [10]:
metro_trash=metro_trash.rename(columns={'Request Number': 'Request_id', 'Date Opened' :'dt_opened', 
                                            'Incident Address': 'Address', 'Zip Code': 'Zip', 'Trash Hauler': 'Hauler'
                                           ,'Council District': 'Council_dist' })

In [11]:
metro_trash.shape

(20226, 11)

In [12]:
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Zip,Hauler,Trash Route,Council_dist,State Plan X,State Plan Y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [13]:
metro_trash=pd.merge(metro_trash, metro, on='Request_id', how='inner')

In [14]:
metro_trash.head()

,Request_id,dt_opened_x,Request,Description,Address_x,Zip_x,Hauler,Trash Route,Council_dist_x,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,Additional_subrequest,dt_opened_y,dt_closed,Parent Request,Closed When Created,Address_y,City,Council_dist_y,Zip_y,Lat,Long,Location
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Backdoor,11/1/17 7:09,11/2/17 16:02,NaN,True,3817 Crouch Dr,NASHVILLE,2.0,37207,36.21730,-86.81540,"(36.2173, -86.8154)"
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 7:42,11/2/17 16:02,NaN,True,4028 Clarksville Pike,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)"
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 7:53,11/2/17 16:00,NaN,True,6528 Thunderbird Dr,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)"
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 8:28,11/7/17 4:03,NaN,True,2603 old matthews rd,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)"
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 8:55,11/2/17 16:00,NaN,True,604 croley dr,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)"


In [15]:
metro_trash.shape

(20226, 26)

In [16]:
metro_trash=metro_trash.drop(columns=['Additional_subrequest', 'Council_dist_x', 'Zip_x', 'dt_opened_y', 'Address_y'])

In [17]:
metro_trash=metro_trash.rename(columns={'dt_opened_x': 'dt_opened', 'Request ': 'Request', 'Address_x': 'Address',
                                            'Zip_y': 'Zip', 'Trash Route': 'Route'})

In [18]:
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,RED RIVER,3205,1.727970e+06,686779.478089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,2.0,37207,36.21730,-86.81540,"(36.2173, -86.8154)"
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)"
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)"
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,WASTE IND,2206,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)"
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)"


In [19]:
metro_trash.Request.value_counts

<bound method IndexOpsMixin.value_counts of 0                            Trash - Backdoor
1        Trash - Curbside/Alley Missed Pickup
2        Trash - Curbside/Alley Missed Pickup
3        Trash - Curbside/Alley Missed Pickup
4        Trash - Curbside/Alley Missed Pickup
                         ...                 
20221    Trash - Curbside/Alley Missed Pickup
20222    Trash - Curbside/Alley Missed Pickup
20223    Trash - Curbside/Alley Missed Pickup
20224    Trash - Curbside/Alley Missed Pickup
20225    Trash - Curbside/Alley Missed Pickup
Name: Request, Length: 20226, dtype: object>

In [20]:
metro_trash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20226 entries, 0 to 20225
Data columns (total 21 columns):
Request_id             20226 non-null int64
dt_opened              20226 non-null datetime64[ns]
Request                20226 non-null object
Description            20197 non-null object
Address                20217 non-null object
Hauler                 19325 non-null object
Route                  19279 non-null object
State Plan X           20198 non-null float64
State Plan Y           20198 non-null float64
Status                 20226 non-null object
Request_type           20226 non-null object
Subrequest_type        20226 non-null object
dt_closed              20131 non-null object
Parent Request         116 non-null float64
Closed When Created    20226 non-null bool
City                   20178 non-null object
Council_dist_y         20177 non-null float64
Zip                    20151 non-null object
Lat                    20201 non-null float64
Long                   20201

In [21]:
metro_trash['Request']=metro_trash['Request'].astype(str)
metro_trash['Description']=metro_trash['Description'].astype(str)
metro_trash['Req_desc']=metro_trash['Request']+metro_trash['Description']
metro_trash=metro_trash[metro_trash['Req_desc'].str.contains('[Mm]iss', regex=True, na=False)]



Standardize the address column

In [22]:
metro_trash['Address']=metro_trash['Address'].astype(str)
metro_trash['Address']=metro_trash['Address'].str.lower()
metro_trash['new_add']=metro_trash['Address'].str.replace(r'(nashville)', r',nashville')
metro_trash['new_add']=metro_trash['new_add'].replace(regex=True, to_replace=r'\,.+', value=r'').str.strip()
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'drive', r'dr')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(road)',r'rd')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(boulevard)',r'blvd')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(court)',r'ct')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(trace)',r'trce')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(avenue)',r'ave')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(circle)',r'cir')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(street)',r'st')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(place)',r'pl')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(pike|pke)',r'pk')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(lane)',r'ln')
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location,Req_desc,new_add
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 clarksville pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)",Trash - Curbside/Alley Missed PickupCurb/Trash...,4028 clarksville pk
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 thunderbird dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)",Trash - Curbside/Alley Missed PickupCurb/trash...,6528 thunderbird dr
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,WASTE IND,2206,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)",Trash - Curbside/Alley Missed Pickupmissed,2603 old matthews rd
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)",Trash - Curbside/Alley Missed PickupMissed the...,604 croley dr
7,25327,2017-11-01 09:32:00,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 jo johnston ave,METRO,9208,1.731459e+06,666013.601229,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,11/2/17 12:03,NaN,True,NASHVILLE,21.0,37203,36.16034,-86.80299,"(36.16034, -86.80299)","Trash Collection ComplaintTrash out on time, m...",1816 jo johnston ave


How many misses are duplicates?

In [23]:
metro_trash.Route = [str(x) if type(x) != float else x for x in metro_trash.Route]
metro_trash['pickup_day'] = [int(x[1]) if type(x) != float else x for x in metro_trash.Route]
metro_trash['weekday'] = [x.weekday() for x in metro_trash.dt_opened]
metro_trash["differ"] = (metro_trash.weekday - metro_trash.pickup_day + 1) % 7


def pickup_date():
   p_date = []
   for x in metro_trash.dt_opened:
     for y in metro_trash.differ:
       if math.isnan(y):
           a = y
       else:
           a = x - timedelta(days = y)
       break
     p_date.append(a)
   return p_date


metro_trash['pickup_date'] = pickup_date()
dedup_miss = metro_trash.groupby('pickup_date').first().reset_index()
dedup_miss

,pickup_date,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location,Req_desc,new_add,pickup_day,weekday,differ
0,2017-10-31 07:42:00,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 clarksville pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.213480,-86.838110,"(36.21348, -86.83811)",Trash - Curbside/Alley Missed PickupCurb/Trash...,4028 clarksville pk,2.0,2,1.0
1,2017-10-31 07:53:00,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 thunderbird dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.142930,-86.885570,"(36.14293, -86.88557)",Trash - Curbside/Alley Missed PickupCurb/trash...,6528 thunderbird dr,2.0,2,1.0
2,2017-10-31 08:28:00,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,WASTE IND,2206,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,NASHVILLE,2.0,37207,36.212660,-86.789180,"(36.21266, -86.78918)",Trash - Curbside/Alley Missed Pickupmissed,2603 old matthews rd,2.0,2,1.0
3,2017-10-31 08:55:00,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.154870,-86.875000,"(36.15487, -86.875)",Trash - Curbside/Alley Missed PickupMissed the...,604 croley dr,2.0,2,1.0
4,2017-10-31 09:32:00,25327,2017-11-01 09:32:00,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 jo johnston ave,METRO,9208,1.731459e+06,666013.601229,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,11/2/17 12:03,NaN,True,NASHVILLE,21.0,37203,36.160340,-86.802990,"(36.16034, -86.80299)","Trash Collection ComplaintTrash out on time, m...",1816 jo johnston ave,2.0,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16680,2019-10-31 16:03:00,267125,2019-11-01 16:03:00,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 murfreesboro pike,RED RIVER,4502,1.781137e+06,632448.551144,In Progress,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,NaN,NaN,False,ANTIOCH,32.0,37013,36.069130,-86.633970,"(36.06913, -86.63397)",Trash - Curbside/Alley Missed PickupMISSED...N...,2731 murfreesboro pk,5.0,4,0.0
16681,2019-10-31 16:08:00,267126,2019-11-01 16:08:00,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 long ave, nashville, tn 37206, united states",METRO,9508,1.749711e+06,669201.601569,In Progress,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,NaN,NaN,False,NASHVILLE,6.0,37206,36.169455,-86.741186,"(36.1694553, -86.7411856)",Trash - Curbside/Alley Missed Pickupentire alley,1621 long ave,5.0,4,0.0
16682,2019-10-31 16:13:00,267130,2019-11-01 16:13:00,Trash - Curbside/Alley Missed Pickup,missed several,"2943 windemere cir, nashville, tn 37214, unite...",RED RIVER,1502,1.770293e+06,674936.303809,In Progress,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,NaN,NaN,False,NASHVILLE,15.0,37214,36.185644,-86.671752,"(36.18564399999999, -86.67175220000001)",Trash - Curbside/Alley Missed Pickupmissed sev...,2943 windemere cir,5.0,4,0.0
16683,2019-10-31 16:15:00,267134,2019-11-01 16:15:00,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 murfreesboro pike, nashville, tn 37013, u...",RED RIVER,4502,1.785225e+06,627146.400187,In Progress,"Trash, Recycling & Litter",Missed Pickup - Tr

metro_trash['Route']=metro_trash['Route'].astype(str)
metro_trash['rtpick_up']=metro_trash['Route'].str.get(1)
metro_trash.head()

metro_trash['dt_opened_month'] = metro_trash['dt_opened'].dt.month
metro_trash['dt_opened_yr'] = metro_trash['dt_opened'].dt.year
metro_trash['dt_opened_dt']= metro_trash['dt_opened'].dt.day
metro_trash['day_opened']=metro_trash['dt_opened'].dt.weekday


In [24]:
metro_trash.isnull().any()
nan_rows = metro_trash[metro_trash['Hauler'].isnull()]
nan_rows

,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location,Req_desc,new_add,pickup_day,weekday,differ,pickup_date
80,25864,2017-11-03 09:19:00,Trash Collection Complaint,missed pick up - please call (615) 519-3192 wa...,115 rosa l parks blvd,NaN,NaN,1.737458e+06,665597.600905,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,11/3/17 17:02,NaN,False,NASHVILLE,19.0,37203,36.159330,-86.782660,"(36.15933, -86.78266)",Trash Collection Complaintmissed pick up - ple...,115 rosa l parks blvd,NaN,4,NaN,2017-11-02 09:19:00
156,26674,2017-11-07 13:40:00,Trash - Curbside/Alley Missed Pickup,customer says trash is not being picked up bet...,111 2nd ave n,NaN,NaN,1.739543e+06,666600.801765,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 16:46,NaN,True,NASHVILLE,19.0,37201,36.162130,-86.775620,"(36.16213, -86.77562)",Trash - Curbside/Alley Missed Pickupcustomer s...,111 2nd ave n,NaN,1,NaN,2017-11-06 13:40:00
161,26746,2017-11-07 16:36:00,Trash - Curbside/Alley Missed Pickup,Missed.,824 harvest grove dr,NaN,NaN,1.789788e+06,626552.764704,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 16:38,NaN,True,ANTIOCH,33.0,37013,36.053090,-86.604580,"(36.05309, -86.60458)",Trash - Curbside/Alley Missed PickupMissed.,824 harvest grove dr,NaN,1,NaN,2017-11-06 16:36:00
165,26847,2017-11-08 09:14:00,Trash Collection Complaint,entire street was missed yesterday 11/07/17,3550 pewitt rd,NaN,NaN,1.721976e+06,687846.927251,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,11/8/17 9:54,NaN,True,NASHVILLE,1.0,37218,36.220090,-86.835750,"(36.22009, -86.83575)",Trash Collection Complaintentire street was mi...,3550 pewitt rd,NaN,2,NaN,2017-11-07 09:14:00
320,28848,2017-11-17 16:33:00,Trash - Curbside/Alley Missed Pickup,Had trash at road last night but still missed ...,4049 moss rd.,NaN,NaN,1.772182e+06,628840.169898,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/17/17 16:44,NaN,True,ANTIOCH,28.0,37013,36.059060,-86.664190,"(36.05906, -86.66419)",Trash - Curbside/Alley Missed PickupHad trash ...,4049 moss rd.,NaN,4,NaN,2017-11-16 16:33:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20057,264970,2019-10-28 12:20:00,Trash - Curbside/Alley Missed Pickup,I filed a missed pick-up on Thursday evening. ...,"543 richmar dr, nashville, tn 37211, united st...",NaN,NaN,1.744449e+06,636430.397628,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,10/28/19 18:32,NaN,False,NASHVILLE,26.0,37211,36.079310,-86.758226,"(36.0793098, -86.75822640000001)",Trash - Curbside/Alley Missed PickupI filed a ...,543 richmar dr,NaN,0,NaN,2019-10-27 12:20:00
20156,266472,2019-10-31 12:25:00,Trash - Curbside/Alley Missed Pickup,miss/curb Trash Tuesday,"1816 glade st, nashville, tn 37207, united states",NaN,NaN,1.736967e+06,681593.540683,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,10/31/19 17:25,NaN,False,NASHVILLE,2.0,37207,36.203285,-86.784787,"(36.2032855, -86.78478659999996)",Trash - Curbside/Alley Missed Pickupmiss/curb ...,1816 glade st,NaN,3,NaN,2019-10-30 12:25:00
20187,266746,2019-11-01 08:48:00,Trash - Curbside/Alley Missed Pickup,test,"750 s 5th st, nashville, tn 37206, united states",NaN,NaN,1.743762e+06,667573.600089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/1/19 15:31,NaN,False,NASHVILLE,6.0,37206,36.163681,-86.760374,"(36.1636811, -86.76037439999999)",Trash - Curbside/Alley Missed Pickuptest,750 s 5th st,NaN,4,NaN,2019-10-31 08:48:00
20189,266771,2019-11-01 09:16:00,Trash - Curbside/Alley Missed Pickup,missed street,643 e campbell rd,NaN,3412,1.753544e+06,708009.199819,In Progress,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,NaN,NaN,False,MADISON,8

In [25]:
metro_trash.isnull().sum()
#https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe/29530559

Request_id                 0
dt_opened                  0
Request                    0
Description                0
Address                    0
Hauler                   729
Route                    749
State Plan X              25
State Plan Y              25
Status                     0
Request_type               0
Subrequest_type            0
dt_closed                 81
Parent Request         17225
Closed When Created        0
City                      40
Council_dist_y            41
Zip                       61
Lat                       23
Long                      23
Location                  23
Req_desc                   0
new_add                    0
pickup_day               749
weekday                    0
differ                   749
pickup_date                0
dtype: int64

metro_trash['Hauler']=metro_trash['Hauler'].astype(str)
metro_trash['Hauler']=metro_trash[''].str.lower()

In [26]:
# plot the number of haulers
metro_haulers=metro_trash.groupby('Hauler').count()
metro_haulers.head()
#sns.catplot()

,Request_id,dt_opened,Request,Description,Address,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location,Req_desc,new_add,pickup_day,weekday,differ,pickup_date
Hauler,,,,,,,,,,,,,,,,,,,,,,,,,,
METRO,2932,2932,2932,2932,2932,2925,2931,2931,2932,2932,2932,2922,5,2932,2930,2930,2928,2929,2929,2929,2932,2932,2925,2932,2925,2932
Metro,44,44,44,44,44,1,44,44,44,44,44,44,0,44,44,44,44,44,44,44,44,44,1,44,1,44
RED RIVER,12506,12506,12506,12506,12506,12504,12501,12501,12506,12506,12506,12456,46,12506,12493,12493,12478,12501,12501,12501,12506,12506,12504,12506,12504,12506
WASTE IND,1116,1116,1116,1116,1116,1116,1115,1115,1116,1116,1116,1101,5,1116,1113,1113,1112,1116,1116,1116,1116,1116,1116,1116,1116,1116


Note and remove from calculation any overlapping misses from one month to the next so that any particular miss is not charged to the contractor more than once.  At least one miss must be in the current month


How many times entire street is missed?
metro_trash_entire is dataframe that has entire street missed. description that contains entire or whole is used to find the dataframe that has entire street

In [27]:
metro_trash_entire = dedup_miss[dedup_miss['Description'].str.contains(r'whole\s\w+|entire\s\w+|other\s\w+', na=False)]
metro_trash_entire['area'] = [re.search(r'\d*\s?(.+)', x).group(1) for x in metro_trash_entire.Address]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
metro_trash_entire.area.value_counts()

wild oaks ct                                            13
barton ln, nashville, tn 37214, united states            9
old hickory blvd, nashville, tn 37221, united states     9
devon dr, nashville, tn 37220, united states             8
buena vista pike, nashville, tn 37218, united states     8
                                                        ..
college ave, 37209                                       1
idlewild pl, nashville, tn 37115, united states          1
old tusculum rd, nashville, tn 37013, united states      1
coastal ct e                                             1
flintlock ct                                             1
Name: area, Length: 1200, dtype: int64

In [29]:
metro_trash_entire['month_year'] = metro_trash_entire.dt_opened.dt.to_period('M')
metro_trash_entire.groupby(['Hauler', 'month_year', 'area']).first()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


pickup_date  \
Hauler    month_year area                                                                 
METRO     2017-11    franklin ave                                   2017-11-05 10:18:00   
                     iverson ave                                    2017-11-02 16:30:00   
                     park cir                                       2017-11-21 15:09:00   
          2017-12    ashwood ave                                    2017-12-20 10:12:00   
                     blue ridge dr                                  2017-12-28 12:17:00   
...                                                                                 ...   
WASTE IND 2019-10    32nd ave n, nashville, tn 37209, united states 2019-10-09 10:31:00   
                     3rd ave n, nashville, tn 37208, united states  2019-10-14 20:47:00   
                     5th ave n, nashville, tn 37208, united states  2019-10-03 16:23:00   
                     6th ave n, nashville, tn 37208, united states  2019-10-17 12:14:00   
                     martin st, nashville, tn 37203, united states  2019-10-13 10:59:00   

                                                                     Request_id  \
Hauler    month_year area                                                         
METRO     2017-11    franklin ave                                         26233   
                     iverson ave                                          26038   
                     park cir                                             29677   
          2017-12    ashwood ave                                          34633   
                     blue ridge dr                                        35835   
...                                                                         ...   
WASTE IND 2019-10    32nd ave n, nashville, tn 37209, united states      257577   
                     3rd ave n, nashville, tn 37208, united states       259664   
                     5th ave n, nashville, tn 37208, united states       255657   
                     6th ave n, nashville, tn 37208, united states       261348   
                     martin st, nashville, tn 37203, united states       258825   

                                                                              dt_opened  \
Hauler    month_year area                                                                 
METRO     2017-11    franklin ave                                   2017-11-06 10:18:00   
                     iverson ave                                    2017-11-03 16:30:00   
                     park cir                                       2017-11-22 15:09:00   
          2017-12    ashwood ave                                    2017-12-21 10:12:00   
                     blue ridge dr                                  2017-12-29 12:17:00   
...                                                                                 ...   
WASTE IND 2019-10    32nd ave n, nashville, tn 37209, united states 2019-10-10 10:31:00   
                     3rd ave n, nashville, tn 37208, united states  2019-10-15 20:47:00   
                     5th ave n, nashville, tn 37208, united states  2019-10-04 16:23:00   
                     6th ave n, nashville, tn 37208, united states  2019-10-18 12:14:00   
                     martin st, nashville, tn 37203, united states  2019-10-14 10:59:00   

                                                                                                  Request  \
Hauler    month_year area                                                                                   
METRO     2017-11    franklin ave                                    Trash - Curbside/Alley Missed Pickup   
                     iverson ave                                     Trash - Curbside/Alley Missed Pickup   
                     park cir                                        Trash - Curbside/Alley Missed Pickup   
          2017-12    ashwood ave                                     Trash - Curbsi

only doing on red river for rolling window. later can expand to the whole dataframe

In [35]:
metro_redriver=dedup_miss[(dedup_miss['Hauler'] == 'RED RIVER')]
metro_redriver.head()

,pickup_date,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location,Req_desc,new_add,pickup_day,weekday,differ
0,2017-10-31 07:42:00,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 clarksville pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)",Trash - Curbside/Alley Missed PickupCurb/Trash...,4028 clarksville pk,2.0,2,1.0
1,2017-10-31 07:53:00,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 thunderbird dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)",Trash - Curbside/Alley Missed PickupCurb/trash...,6528 thunderbird dr,2.0,2,1.0
3,2017-10-31 08:55:00,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)",Trash - Curbside/Alley Missed PickupMissed the...,604 croley dr,2.0,2,1.0
5,2017-10-31 09:45:00,25330,2017-11-01 09:45:00,Trash - Curbside/Alley Missed Pickup,Missed.,4484 lavergne couchville pike,RED RIVER,4210,1.794534e+06,618749.342732,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:04,NaN,True,ANTIOCH,33.0,37013,36.03173,-86.58837,"(36.03173, -86.58837)",Trash - Curbside/Alley Missed PickupMissed.,4484 lavergne couchville pk,2.0,2,1.0
6,2017-10-31 10:07:00,25341,2017-11-01 10:07:00,Trash - Curbside/Alley Missed Pickup,Missed trash pickup - resident has at curb for...,3113 hydes ferry rd,RED RIVER,4204,1.721578e+06,676018.399900,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,2.0,37218,36.18759,-86.83676,"(36.18759, -86.83676)",Trash - Curbside/Alley Missed PickupMissed tra...,3113 hydes ferry rd,2.0,2,1.0


calculate the misses every month. 
for this, use the dataframe dedup_miss in which duplicate misses are removed
look at 

In [38]:
metro_redriver.groupby(by='Address').rolling(window= '30D').Request_id.count()

ValueError: window must be an integer

metro_trash.groupby(['dt_opened_month', 'dt_opened_yr', 'pick_up', 'dt_opened_day', 'Address'])
accre_df.groupby(by='account').rolling(on = 'submit_time', window = '4h').jobid.count()